# Dependency Injection in Python with FastAPI

## 1. Introduction to Dependency Injection

Dependency Injection (DI) is a design pattern that allows the decoupling of object creation from object usage, promoting more maintainable, testable, and flexible code. In this technique the object's dependencies are provided by an external entity, rather than the object creating them itself. This approach allows for greater flexibility and easier testing.

### A Simple Example

**Without DI**

In [1]:
class Database:
    def connect(self):
        print("Connecting to the database")

class Service:
    def __init__(self):
        self.db = Database()

    def perform_action(self):
        self.db.connect()
        print("Performing an action")

service = Service()
service.perform_action()

Connecting to the database
Performing an action


In this example, the `Service` class is tightly coupled with the `Database` class, making it difficult to change the database implementation or to test the `Service` class in isolation.

**With DI**

In [2]:
class Database:
    def connect(self):
        print("Connecting to the database")

class Service:
    def __init__(self, db: Database):
        self.db = db

    def perform_action(self):
        self.db.connect()
        print("Performing an action")

db = Database()
service = Service(db)
service.perform_action()

Connecting to the database
Performing an action


Here, the `Service` class receives its `Database` dependency through the constructor, allowing for greater flexibility and easier testing.

## 2. Benefits of Dependency Injection

1. **Decoupling**: Reduces the tight coupling between classes.
2. **Flexibility**: Easier to swap implementations of dependencies.
3. **Testability**: Facilitates testing by allowing the injection of mock dependencies.
4. **Maintainability**: Promotes cleaner and more maintainable code.

## 3. Basic Concepts

- **Dependency**: An object that another object relies on.
- **Injection**: The process of providing a dependency to an object.
- **Inversion of Control (IoC)**: A principle where the control of object creation and management is transferred from the object itself to an external entity.

## 4. DI in Python

In Python, DI can be achieved using various techniques like constructor injection, method injection, and property injection. We'll explore these techniques with examples.

### Constructor Injection

Constructor injection involves passing dependencies through the constructor.

In [3]:
class Logger:
    def log(self, message: str):
        print(f"LOG: {message}")

class UserService:
    def __init__(self, logger: Logger):
        self.logger = logger

    def create_user(self, username: str):
        self.logger.log(f"User {username} created")

logger = Logger()
user_service = UserService(logger)
user_service.create_user("Alice")

LOG: User Alice created


### Method Injection

Method injection involves passing dependencies through methods.

In [4]:
class PaymentProcessor:
    def process(self, amount: float):
        print(f"Processing payment of {amount}")

class OrderService:
    def create_order(self, processor: PaymentProcessor, amount: float):
        processor.process(amount)
        print("Order created")

processor = PaymentProcessor()
order_service = OrderService()
order_service.create_order(processor, 100.0)

Processing payment of 100.0
Order created


### Property Injection

Property injection involves setting dependencies through properties.

In [5]:
class EmailService:
    def send_email(self, to: str, subject: str, body: str):
        print(f"Sending email to {to} with subject {subject}")

class NotificationService:
    def __init__(self):
        self.email_service = None

    def notify(self, message: str):
        if self.email_service:
            self.email_service.send_email("user@example.com", "Notification", message)
        else:
            print("Email service not available")

email_service = EmailService()
notification_service = NotificationService()
notification_service.email_service = email_service
notification_service.notify("This is a test notification")

Sending email to user@example.com with subject Notification


## 5. DI in FastAPI

FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.6+ based on standard Python type hints. It supports Dependency Injection out of the box, making it easy to manage dependencies in web applications.

I'll create a simple FastAPI application demonstrating DI. This will not run in Jupyter notebook but you may put it into a Python app to see the results. Here it is put only for demonstartion purposes.

First, install FastAPI and Uvicorn:

In [13]:
# ! pip install fastapi uvicorn typer==0.9.0 -q

In [ ]:
# app.py
from fastapi import FastAPI, Depends

app = FastAPI()

class Database:
    def __init__(self):
        self.connection = "Database connection established"

def get_database():
    db = Database()
    try:
        yield db
    finally:
        print("Closing database connection")

@app.get("/items/")
def read_items(db: Database = Depends(get_database)):
    return {"message": db.connection}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

In this example:
- We define a `Database` class to simulate a database connection.
- The `get_database` function is a dependency provider that yields a `Database` instance.
- The `read_items` endpoint depends on the `get_database` function, which injects the `Database` instance into the endpoint.

To run the application, use:

In [12]:
# ! uvicorn app:app --reload

Visit `http://127.0.0.1:8000/items/` to see the response.

## 6. Testing with Dependency Injection

DI makes testing easier by allowing the injection of mock dependencies. You may use the below test with the above example. Here it is put only for demonstartion purposes.

In [ ]:
from fastapi.testclient import TestClient

class MockDatabase(Database):
    def __init__(self):
        self.connection = "Mock database connection"

def get_mock_database():
    db = MockDatabase()
    try:
        yield db
    finally:
        print("Closing mock database connection")

app.dependency_overrides[Database] = get_mock_database

client = TestClient(app)

def test_read_items():
    response = client.get("/items/")
    assert response.status_code == 200
    assert response.json() == {"message": "Mock database connection"}

In this example:
- We create a `MockDatabase` class to simulate the database for testing.
- The `get_mock_database` function provides the `MockDatabase` instance.
- We override the dependency in the `app` with `get_mock_database` for testing.
- The `test_read_items` function tests the `/items/` endpoint using the mock database.

## 7. Conclusion

Dependency Injection is a powerful design pattern that promotes decoupling, flexibility, and testability. FastAPI provides excellent support for DI, making it easy to manage dependencies in web applications. By understanding and implementing DI, you can write more maintainable and testable code.